## Data Gathering

In [1]:
#Load library
import pandas as pd 
import numpy as np
import requests
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import tweepy
%matplotlib inline

### Load archived twitter data from csv file

In [2]:
#Read @WeRateDogs twitter acrhive 
df_archive = pd.read_csv("twitter-archive-enhanced.csv")

In [84]:
df_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [11]:
df_archive.tweet_id[1]

892177421306343426

### Load more information using twitter apis

In [3]:
#Connect Twitter APIs to query more data related to archived tweets
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'


In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [32]:
df_archive.shape[0]

2356

In [14]:
fails_dict = {}
tweet_info_bulk = []
for n in range(0,df_archive.shape[0]+100,100):
    if df_archive.shape[0] - n < 100:
        tweet_id = df_archive.tweet_id[n:df_archive.shape[0]].tolist()
    else:
        tweet_id = df_archive.tweet_id[n:n+100].tolist()
    try:
        lookups = api.statuses_lookup(id_=tweet_id)
        for info in lookups:
            tweet_info_bulk.append({"tweet_id": info.id,
                            "favorite_count": info.favorite_count,
                            "content": info.text})
    except tweepy.TweepError as e:
        fails_dict[e] = tweet_id
#     except tweepy.TweepError as e:
#         print("fail")
#         fails_dict[tweet.id] = e

In [17]:
with open("./tweet_info.json", "w+") as file:
    json.dump(tweet_info_bulk, file)

In [18]:
df_tweet_info = pd.read_json("./tweet_info.json",)

In [19]:
df_tweet_info.shape

(2333, 3)

In [20]:
df_tweet_info.head()

,content,favorite_count,tweet_id
0,This is Benedict. He wants to thank you for th...,18048,876484053909872640
1,This is Roscoe. Another pupper fallen victim t...,20298,886366144734445568
2,RT @rachel2195: @dog_rates the boyfriend and h...,0,877611172832227328
3,This is Gerald. He was just told he didn't get...,20862,888078434458587136
4,This is Darla. She commenced a snooze mid meal...,40415,891689557279858688


### Load image prediction data from tsv file 

In [37]:
df_img_predict = pd.read_csv("image-predictions.tsv", delimiter='\t')

## Data Assessing 

In [33]:
df_archive.sample(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1921,674262580978937856,NaN,NaN,2015-12-08 16:21:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Gus. He's super stoked about being an ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/674262580...,9,10,Gus,None,None,pupper,None
569,801958328846974976,NaN,NaN,2016-11-25 01:18:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Ronnie. He hopes you're having a great...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/801958328...,12,10,Ronnie,None,None,None,None
1215,715009755312439296,NaN,NaN,2016-03-30 02:56:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Siba. She's remarkably mobile. Very sl...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/715009755...,12,10,Siba,None,None,None,None
1048,743253157753532416,NaN,NaN,2016-06-16 01:25:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Kilo. He cannot reach the snackum. Nif...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/743253157...,10,10,Kilo,None,None,None,puppo
53,881906580714921986,NaN,NaN,2017-07-03 16:04:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Rey. He's a Benebop Cumberfloof. 12/10...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/881906580...,12,10,Rey,None,None,None,None
1265,709901256215666688,NaN,NaN,2016-03-16 00:37:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",WeRateDogs stickers are here and they're 12/10...,NaN,NaN,NaN,"http://goo.gl/ArWZfi,https://twitter.com/dog_r...",12,10,None,None,None,None,None
409,823333489516937216,8.233264e+17,1.582854e+09,2017-01-23 00:56:15 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@HistoryInPics 13/10,NaN,NaN,NaN,NaN,13,10,None,None,None,None,None
1219,714631576617938945,NaN,NaN,2016-03-29 01:53:39 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Benedict. He's a feisty pup. Needs a b...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/714631576...,4,10,Benedict,None,None,None,None
371,828408677031882754,NaN,NaN,2017-02-06 01:03:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bear. He went outside to play in the s...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/828408677...,12,10,Bear,None,None,None,None
2092,670782429121134593,NaN,NaN,2015-11-29 01:52:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This dude slaps your girl's ass what do you do...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670782429...,5,10,None,None,None,None,None


In [38]:
df_img_predict.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [39]:
df_tweet_info.head()

,content,favorite_count,tweet_id
0,This is Benedict. He wants to thank you for th...,18048,876484053909872640
1,This is Roscoe. Another pupper fallen victim t...,20298,886366144734445568
2,RT @rachel2195: @dog_rates the boyfriend and h...,0,877611172832227328
3,This is Gerald. He was just told he didn't get...,20862,888078434458587136
4,This is Darla. She commenced a snooze mid meal...,40415,891689557279858688


### Data Quality Issues

#### df_archive 
- There are some retweet data without dog ratings
- Lots of unnecessary columns from "archive data' such as ["in_reply_to_status_id","in_reply_to_user_id","retweeted_status_id","retweeted_status_user_id","retweeted_status_timestamp","source","expanded_urls"]
- Numerator 13.5 is extracted as 5 and data type is integer
- Extract the number of legs as rating 3 1/2 legged 
- Dog names are not properly extracted i.g. a, the, an
- There are None in dog_stage columns
- "Twitter_id" datatype is Integer
- Timestamp colum is not "datetime" type

#### df_img_predict

- A mix of uppercase and lowercase in p1, p2, p3 columns of image prediction table

### Data Tideness Issues 
- All 3 dataframs are all seperate
- Dog stage values are scattered over 4 columns ['doggo','floofer','pupper','puppo']


## Data Cleaning 

In [42]:
#create a copy for each dataframe
archive_copy = df_archive.copy()
tweet_info_copy = df_tweet_info.copy()
img_predict_copy = df_img_predict.copy()

### Define
Only include the rows without any values in in_reply_to_status_id or retweeted_status_id


### Code